### Calculation of polaron migration barriers using linear interpolation
In this example we show how to calculate polaron migration barrier on the example of LiFePO4


In [11]:
from siman.calc_manage import smart_structure_read
from siman.geo import interpolate


In [9]:
st = smart_structure_read('in/POSCAR_LiFePO4_1u_1_end') #read LiFePO4 structure
# st.printme()

In [10]:
st1 = st.localize_polaron(26,  -0.2) # displace surrounding oxygens by 0.2 A in the direction of iron, corresponding to oxidation
st2 = st.localize_polaron(27, -0.2)
st1.write_poscar() # write POSCAR to check in jmol that the correct displacement was performed
st2.write_poscar()

-- POSCAR was written to /hdd/home/aksenov/Simulation_wrapper/siman/tutorials/xyz/POSCAR__LiFePO4_1u_1_endpol27 

-- POSCAR was written to /hdd/home/aksenov/Simulation_wrapper/siman/tutorials/xyz/POSCAR__LiFePO4_1u_1_endpol28 



'/hdd/home/aksenov/Simulation_wrapper/siman/tutorials/xyz/POSCAR__LiFePO4_1u_1_endpol28'

In [12]:
# *images* is the number of intermediate structures
# *write_poscar* write files to folder *poscar_folder*  
interpolate(st1, st2, images = 10, write_poscar = 1, omit_edges = 0, poscar_folder = 'xyz/' ) # create intermediate structures by linerar interpolation between st1 and st2

-- POSCAR was written to xyz/1.POSCAR 

-- POSCAR was written to xyz/2.POSCAR 

-- POSCAR was written to xyz/3.POSCAR 

-- POSCAR was written to xyz/4.POSCAR 

-- POSCAR was written to xyz/5.POSCAR 

-- POSCAR was written to xyz/6.POSCAR 

-- POSCAR was written to xyz/7.POSCAR 

-- POSCAR was written to xyz/8.POSCAR 

-- POSCAR was written to xyz/9.POSCAR 

-- POSCAR was written to xyz/10.POSCAR 

-- POSCAR was written to xyz/11.POSCAR 

-- POSCAR was written to xyz/12.POSCAR 



1. Check all intermediate images by opening them simulteniously in Jmol and clicking next frame
2. Make a shell script that launches VASP for:
   a) for 1.POSCAR, 2.POSCAR ... 12.POSCAR in a loop (forward branch)
   b) for 12.POSCAR 11.POSCAR ... 1.POSCAR in a loop (backward branch)
3. Plot Total energy vs structure number for forward and backward branches on one figure
4. Make sure that  ISTART	= 1; LWAVE  =  .TRUE.; ICHARG = 0; in your INCAR 